# PROJECT : IMAGE CAPTIONING 

## Importing the libreries

In [4]:
import sys
sys.path.append("..")
import grading
import download_utils

import tensorflow as tf
import keras
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
L = keras.layers
K = keras.backend
import utils
import time
import zipfile
import json
from collections import defaultdict
import re
import random
from random import choice
import grading_utils
import os
from keras_utils import reset_tf_session
import tqdm_utils

## Extracting the image features

In [12]:
IMG_SIZE = 299

# we take the last hidden layer of IncetionV3 as an image embedding

def get_cnn_encoder():
    
    K.set_learning_phase(False)
    model = keras.applications.InceptionV3(include_top = False)
    preprocess_for_model = keras.applications.inception_v3.preprocess_input
    
    model = keras.models.Model(model.inputs , keras.layers.GlobalAveragePooling2D()(model.output))
    
    return model , preprocess_for_model

In [ ]:
#load pre-trained model
tf.compat.v1.Session()
encoder, preprocess_for_model = get_cnn_encoder()

#extract train features
train_img_embeds, train_img_fns = utils.apply_model(
    "train2014.zip", encoder, preprocess_for_model, input_shape=(IMG_SIZE, IMG_SIZE))
utils.save_pickle(train_img_embeds, "train_img_embeds.pickle")
utils.save_pickle(train_img_fns, "train_img_fns.pickle")

#extract validation features
val_img_embeds, val_img_fns = utils.apply_model(
    "val2014.zip", encoder, preprocess_for_model, input_shape=(IMG_SIZE, IMG_SIZE))
utils.save_pickle(val_img_embeds, "val_img_embeds.pickle")
utils.save_pickle(val_img_fns, "val_img_fns.pickle")

#sample images for learners
def sample_zip(fn_in, fn_out, rate=0.01, seed=42):
    np.random.seed(seed)
    with zipfile.ZipFile(fn_in) as fin, zipfile.ZipFile(fn_out, "w") as fout:
        sampled = filter(lambda _: np.random.rand() < rate, fin.filelist)
        for zInfo in sampled:
            fout.writestr(zInfo, fin.read(zInfo))
            
sample_zip("E:\Datasets\COCO\train2014.zip", "train2014_sample.zip")
sample_zip("E:\Datasets\COCO\val2014.zip", "val2014_sample.zip")

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\limon\anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\limon\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\limon\Downloads\Nothing\coursera\Try inro-to-dl\week6\utils.py", line 55, in reading_thread
    zf = zipfile.ZipFile(zip_fn)
  File "C:\Users\limon\anaconda3\lib\zipfile.py", line 1240, in __init__
    self.fp = io.open(file, filemode)
FileNotFoundError: [Errno 2] No such file or directory: 'train2014.zip'



In [ ]:
# load prepared embeddings
train_img_embeds = utils.read_pickle("train_img_embeds.pickle")
train_img_fns = utils.read_pickle("train_img_fns.pickle")
val_img_embeds = utils.read_pickle("val_img_embeds.pickle")
val_img_fns = utils.read_pickle("val_img_fns.pickle")
# check shapes
print(train_img_embeds.shape, len(train_img_fns))
print(val_img_embeds.shape, len(val_img_fns))